In [56]:
import pandas as pd

In [122]:
selectCrime2 = pd.read_csv ("selectCrime2.csv")

In [123]:
selectCrime2 = selectCrime2.drop(columns=['Unnamed: 0','All','geometry','nearestSVPoint'])

Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]
selectCrime2[Vars] = selectCrime2[Vars].apply(lambda x: round((x-x.min())/(x.max()-x.min()),2))

selectCrime2 = pd.melt(selectCrime2, id_vars=['cartodb_id','crimeType','count'])
selectCrime2 = selectCrime2.groupby(["variable","value","crimeType"]).sum("count")
selectCrime2 = selectCrime2.reset_index()
selectCrime2.head()

,variable,value,crimeType,cartodb_id,count
0,building,0.0,Aggravated Assault Firearm,808900,1
1,building,0.0,All Other Offenses,12385357,7
2,building,0.0,Burglary Residential,2375978,9
3,building,0.0,Fraud,8328808,4
4,building,0.0,Narcotic / Drug Law Violations,2800598,2


In [128]:
df = selectCrime2.groupby(["variable","value"]).sum("count").reset_index()
df['crimeType'] = "All"
selectCrime2 = df.append(selectCrime2, ignore_index = True)

In [132]:
selectCrime2

,variable,value,cartodb_id,count,crimeType
0,building,0.00,58301664,54,All
1,building,0.01,503256668,386,All
2,building,0.02,722526307,608,All
3,building,0.03,1431903458,1119,All
4,building,0.04,1589717144,1369,All
...,...,...,...,...,...
14488,wall,1.00,14280604,6,Other Assaults
14489,wall,1.00,779444,1,Robbery Firearm
14490,wall,1.00,2038602,4,Theft from Vehicle
14491,wall,1.00,887524,2,Thefts


In [265]:
selection3 = selectCrime2.loc[(selectCrime2["variable"]=="green")]

chart=alt.Chart(selection3).mark_point(
).encode(x='value:Q',y='count:Q')

chart + chart.transform_regression('value','count').mark_line()

alt.LayerChart(...)

# Regression

In [165]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(9876789)

In [267]:
selectCrimeCI = pd.read_csv ("selectCrime2.csv")
selectCrimeCI = selectCrimeCI.drop(columns=['Unnamed: 0','All','geometry','nearestSVPoint'])
Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]
selectCrimeCI[Vars] = selectCrimeCI[Vars].apply(lambda x: round((x-x.min())/(x.max()-x.min()),2))
selectCrimeCI = pd.melt(selectCrimeCI, id_vars=['cartodb_id','crimeType','count'])
selectCrimeCI = selectCrimeCI.groupby(["variable","value"]).sum("count").reset_index()
selectCrimeCI = selectCrimeCI.drop(columns="cartodb_id")

In [217]:
attempt = selectCrimeCI.pivot(index='value', columns='variable', values=['count']).reset_index().fillna(0)
X = attempt[["count"]].to_numpy()
X = sm.add_constant(X)
y = attempt['value'].to_numpy()

In [219]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.900
Method:                 Least Squares   F-statistic:                     90.65
Date:                Sat, 18 Dec 2021   Prob (F-statistic):           1.50e-42
Time:                        21:10:52   Log-Likelihood:                 102.60
No. Observations:                 101   AIC:                            -183.2
Df Residuals:                      90   BIC:                            -154.4
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8592      0.019     44.154      0.0

In [313]:
regressionOutcome = pd.DataFrame(results.params)
VARS = pd.Series(["intercept"]).append(pd.Series(selectCrimeCI["variable"].unique())).reset_index(drop=True)
lowCI = ["intercept","-6.55e-05","-2.17e-05",'-8.82e-05','-3.51e-06',"-4.78e-06"," -8.82e-05","-8.86e-05","-8.57e-05","-6.28e-06","4.69e-06"]
highCI = ["intercept",'4.69e-06','5.74e-05','-4.02e-05','6.07e-06',"-2.17e-06","-9.89e-06","-4.92e-05","-1.59e-05","3.26e-05","4.1e-05"]
coefficient = regressionOutcome.iloc[:,0]

regressionOutcome["vars"] = VARS
regressionOutcome["lowCI"] = lowCI
regressionOutcome["highCI"] = highCI
regressionOutcome = regressionOutcome[1:]
regressionOutcome["coefficient"] = coefficient
# regressionOutcome = regressionOutcome.rename(columns={"0": "coefficient"}, errors="raise")
regressionOutcome = regressionOutcome.iloc[:,1:5]
regressionOutcome = pd.DataFrame(regressionOutcome)
regressionOutcome

,vars,lowCI,highCI,coefficient
1,building,-6.55e-05,4.69e-06,-0.000030
2,green,-2.17e-05,5.74e-05,0.000018
3,infrastructure,-8.82e-05,-4.02e-05,-0.000064
4,lives,-3.51e-06,6.07e-06,0.000001
5,publicservice,-4.78e-06,-2.17e-06,-0.000003
6,road,-8.82e-05,-9.89e-06,-0.000049
7,sidewalk,-8.86e-05,-4.92e-05,-0.000069
8,sky,-8.57e-05,-1.59e-05,-0.000051
9,transportation,-6.28e-06,3.26e-05,0.000013
10,wall,4.69e-06,4.1e-05,0.000023


In [319]:
coefficient = alt.Chart(regressionOutcome).mark_point(
    filled=True,
    size=50,
    color='black'
).encode(
    x=alt.X('coefficient:Q'),
    y=alt.Y('vars:N')
)

CI = coefficient.mark_errorbar().encode(
    y=alt.Y('vars:N'),
    x="lowCI:Q",
    x2="highCI:Q"
)

coefficient + CI

alt.LayerChart(...)

# Scatter Plot

In [164]:
import altair as alt
from vega_datasets import data
alt.data_transformers.enable('json')
selection = alt.selection_multi(fields=['crimeType'], bind='legend')

alt.Chart(selectCrime2).mark_point(
).encode(
    x=alt.X('value:Q',axis=alt.Axis(ticks=False, domain=False)),
    y=alt.Y('count:Q',axis=alt.Axis(ticks=False, domain=False)),
    color=alt.condition(selection,
                        alt.Color('crimeType:N', scale=alt.Scale(scheme='category20b')),
                        alt.value('lightgray')),   
    opacity=alt.condition(selection, alt.value(0.6), alt.value(0.3))
).properties(height=300,width=300
).add_selection(selection
).facet(
    facet='variable:N',
    columns=2
).resolve_scale(
    y='independent'
).interactive()

alt.FacetChart(...)

In [62]:
import altair as alt
from vega_datasets import data

source = data.barley()

error_bars = alt.Chart(source).mark_errorbar(extent='ci').encode(
  x=alt.X('yield:Q', scale=alt.Scale(zero=False)),
  y=alt.Y('variety:N')
)

points = alt.Chart(source).mark_point(filled=True, color='black').encode(
  x=alt.X('yield:Q', aggregate='mean'),
  y=alt.Y('variety:N'),
)

error_bars + points

alt.LayerChart(...)